In [1]:
import pandas as pd
import numpy as np
import tensorflow as tf
import matplotlib.pyplot as plt
from ml import MLHelper

In [5]:
# Binance API parameters
traded_assets = ['ETH', 'ADA', 'DOGE']#, "BTC"]
stable_assets = ['USDT']
start_date = "Jan 01, 2021"
end_date = "Dec 31, 2021"
time_horizons = ['1hr', '12hr']

# EMA parameters
alpha_values = [0.5]

# labeling parameters
trend_reversal_factors = [0.3, 0.6]
change_tolerances = [0.5, 1]
forward_terms = [5, 8]
break_away_points = [1, 2]

history_length = 128

In [6]:
for traded_asset in traded_assets:
    training_df = pd.DataFrame(columns=['time_horizon', 'change_tolerance', 'forward_term', 'break_away_point', 'alpha', 'trend_reversal_factor', 'val_mean_absolute_error'])

    for stable_asset in stable_assets:
        for time_horizon in time_horizons:
            for change_tolerance in change_tolerances:
                for forward_term in forward_terms:
                    for break_away_point in break_away_points:
                        for alpha_value in alpha_values:
                            for trend_reversal_factor in trend_reversal_factors:
                                df = pd.read_csv(f"data/labeled/{traded_asset}_{stable_asset}_{time_horizon}_{alpha_value}_{trend_reversal_factor}_{change_tolerance}_{forward_term}_{break_away_point}_long_utility.csv")
                                df['change'] = 100 * (df['close'] - df['open']) / df['open']
                                df = df[['change', 'long_utility_context_ema']]

                                mlHelper = MLHelper(df=df, history_length=history_length) 
                                features, labels = mlHelper.create_features_labels()

                                eval_start_point = int(0.7*len(df))
                                test_start_point = int(0.9*len(df))

                                x_train, x_eval, x_test = features[:eval_start_point], features[eval_start_point:test_start_point], features[test_start_point:]
                                y_train, y_eval, y_test = labels[:eval_start_point], labels[eval_start_point:test_start_point],labels[test_start_point:]

                                assert x_train.shape[0] == y_train.shape[0], "first index of x_train.shape != first index of y_train"
                                assert x_eval.shape[0] == y_eval.shape[0], "first index of x_eval.shape != first index of y_eval"
                                assert x_test.shape[0] == y_test.shape[0], "first index of x_test.shape != first index of y_test"

                                history = mlHelper.train(x_train=x_train, y_train=y_train, x_eval=x_eval, y_eval=y_eval)

                                training_df.loc[len(training_df)-1] = [time_horizon, change_tolerance, forward_term, break_away_point, alpha_value, trend_reversal_factor, history['val_mean_absolute_error'][-1]]
    
        training_df.to_csv(f"data/labeled/{traded_asset}_{stable_asset}_full_training.csv")

2023-02-26 20:02:30.550806: I tensorflow/core/grappler/optimizers/custom_graph_optimizer_registry.cc:114] Plugin optimizer for device_type GPU is enabled.
2023-02-26 20:02:30.731237: I tensorflow/core/grappler/optimizers/custom_graph_optimizer_registry.cc:114] Plugin optimizer for device_type GPU is enabled.
2023-02-26 20:02:30.919769: I tensorflow/core/grappler/optimizers/custom_graph_optimizer_registry.cc:114] Plugin optimizer for device_type GPU is enabled.
2023-02-26 20:02:35.044532: I tensorflow/core/grappler/optimizers/custom_graph_optimizer_registry.cc:114] Plugin optimizer for device_type GPU is enabled.
2023-02-26 20:02:35.094404: I tensorflow/core/grappler/optimizers/custom_graph_optimizer_registry.cc:114] Plugin optimizer for device_type GPU is enabled.
2023-02-26 20:03:08.335268: I tensorflow/core/grappler/optimizers/custom_graph_optimizer_registry.cc:114] Plugin optimizer for device_type GPU is enabled.
2023-02-26 20:03:08.457203: I tensorflow/core/grappler/optimizers/cust

In [4]:
training_df

,time_horizon,change_tolerance,forward_term,break_away_point,alpha,trend_reversal_factor,val_mean_absolute_error
-1,1hr,0.5,5,1,0.5,0.3,0.191733
0,1hr,0.5,5,1,0.5,0.6,0.178360
1,1hr,0.5,5,2,0.5,0.3,0.189205
2,1hr,0.5,5,2,0.5,0.6,0.177947
3,1hr,0.5,8,1,0.5,0.3,0.192470
4,1hr,0.5,8,1,0.5,0.6,0.194972
5,1hr,0.5,8,2,0.5,0.3,0.191350
6,1hr,0.5,8,2,0.5,0.6,0.194073
7,1hr,1.0,5,1,0.5,0.3,0.206101
8,1hr,1.0,5,1,0.5,0.6,0.172562
